1. 함수 선언
2. view, pure
3. 가시성
4. payable
5. 생성자 - 한 번 호출된 후 더 이상 호출 x
6. 함수로 제약조건 지정(실행 전 후 다른 코드 실행)
7. 이벤트(잘 사용 안 함)
8. fallback

# 함수

1. 내장함수
2. 사용자 정의함수

    fuction 함수명(인자) 이름 return(dataType) {/코드}

# 생성자

    constructer()

1. 안 만들면 자동 생성
2. 오버로딩 불가능
3. 상속 -> 부모 생성자 호출 후 자식 생성자 호출
4. 생성자는 반환값 x

# view, pure

구분 | 설명
----------|----------
view | 상태를 수정하지 않는 경우
pure | 상태를 수정하지도 않으며, 읽지도 않는 경우

```pure``` 함수는 다음과 같은 경우에 사용할 수 없다.

* 상태 변수(state variables)를 읽는 경우,
* 잔고 ```this.balance``` 또는 ```<address>.balance```를 읽는 경우,
* 전역변수 ```block```, ```tx```, ```msg``` (```msg.sig```, ```msg.data``` 제외)를 읽는 경우,
* ```pure```로 표기되지 않은 함수를 호출하는 경우,
* ```opcodes```로 작성된 인라인(inline) 코드

다음 경우에 해당되는 함수들은 블록체인을 수정하기 때문에 ```view```는 사용할 수 없다.

- 상태 변수를 저장하는 경우 (당연히 블록체인에 쓰는 경우이다)
- 이벤트 발생 (이벤트가 발생하면 로그에 기록이 남겨진다.)
- 다른 컨트랙 생성
- ```selfdestruct``` 컨트랙을 삭제하는 경우
- ```call``` 함수로 송금하는 경우
- ```view``` 또는 ```pure``` 아닌 함수를 호출하는 경우
- low-level calls을 호출하는 경우
- ```opcodes```로 작성된 인라인 코드가 있는 경우

# payable

ether에 입출금 할때 사용됨?

# 가시성

* ```public``` 내, 외부에서 누구나 사용할 수 있다. 가장 개방적이다.
* ```internal``` 기본(default) 가시성으로 내부 또는 상속관계에서만 사용할 수 있다.(블록체인 내부에서 사용하는 함수)
* ```private``` 자신만 사용할 수 있다.
* ```external```은 ```internal```의 반대로서 외부에서만(블록체인 내부가 아닌 웹에서 사용자만 사용하는 함수) 사용할 수 있다. 내부에서 호출할 수 있지만 this 키워드를 이용해서 ```this.f()```형태로 사용한다.

함수 적는 순서
- 생성자
- fallback 함수
- external 함수
- public 함수
- internal 함수
- private 함수


In [1]:
%%writefile FuctionTest.sol

//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract FunctionTest {
    int x;
    constructor() { // 없어도 자동으로 생김, 한 번만 생성됨
        x = 0;
    }
    function incrementX() public {
        x += 1;  // 상태변수 x의 변경
    }
    function doubleX() public {
        X2();  //아래에 internal로 정의. 현재 public에서 호출.
    }
    function divideBy(int by) view public returns(int) {// 읽기만 가능한 view 함수
        return x/by; // 소수를 지원하지 않는다. 0, 1/3 등을 테스트해보자.
    }
    // actually NONE returned
    function getX_() view public returns(int) {
        return x;        
    }
    // pure로 설정 테스트해보자
    function getX() view public returns(int) {
        return x;
    }
    function getBalance() view public returns(uint) {
        return(address(this).balance);// this는 이 컨트랙을 의미 함
    }
    // 'payable'을 붙이면 코드가 없어도 입금가능 (msg.value가 입금된다)
    function deposit() public payable {// 코드가 비워져 있다 msg전역변수를 통해서 금액을 입금 시켜줌
    }
    // 외부에서 사용할 수 없다
    function X2() internal {// 내부에서만 사용
        x *= 2; 
    }
    function getBlockNumber() view public returns(uint) {
        return block.number;
    }


Writing FuctionTest.sol


In [ ]:
!solc-window

In [2]:
%%writefile UnderscoreTest.sol
# 변경자
contract UnderscoreTest {
    string season = "";
    function getSeason() view public returns(string memory) {
        return season;
    }   
    function setWinter() public setSummerAfter { // setWinter()를 setSummerAfter의 _ 위치에서 실행
        season = "winter";// summer -> winter
    }

    modifier setSummerAfter() {// 사후 조건 + 사전조건도 만들 수 있음
        season = "summer";
        _;// 이 자리에서 실행 한다는 의미
    }
}

Writing UnderscoreTest.sol


배포자(contract) = 소유자

제약조건
- onlyOwner: 컨트랙 소유자만 가능하도록 제한한다.
- onlyAfter: 입금시 최소 시간이 지나야 가능하도록 제한.
- minBalance: 출금시 최소 잔고가 있어야 가능
- fullback()사용

In [7]:
%%writefile BankV3.sol

//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract BankV3 {
    address owner;// 배포한 사람 주소
    uint balance;
    uint256 timeToDeposit;
    
    event PrintLog(string);
    event Sent(address from, address to, uint amount );
    constructor() {
        owner = msg.sender;// 전역변수로 배포자 주소를 찾음
        balance = 0;
    }
    fallback() external {// 무조건 external로 외부에서만 부를 수 있음
        emit PrintLog("Fallback called");// test용 emit
    }
    function forwardTo(address payable _receiver) public payable onlyOwner {// 계좌이체(소유주만 보내기 가능)
        // require(msg.sender == owner);
        _receiver.transfer(msg.value);// receiver이 받는 사람
        emit Sent(msg.sender, _receiver, msg.value);
    }
    function getBalance() public view returns(uint, uint) {
        return (balance, address(this).balance);// 변수로 만든 잔고, address로 찾은 변수 
    }
    function deposit(uint amount) public payable onlyAfter {// 10초 후에 거래 가능, 입금
        timeToDeposit = block.timestamp + 10 seconds; //timeToDeposit = now + 10 seconds;
        require(msg.value == amount);
        balance += amount;
    }
    function withdrawAll() public onlyOwner minBalance {// 전액을 출금
        balance -= address(this).balance;
        //require(msg.sender == owner);
        //v0.5.0 msg.sender.transfer(address(this).balance);
        payable(msg.sender).transfer(address(this).balance); 
    }
    function getTimeToDeposit() view public returns(uint256) { return timeToDeposit; }
    modifier onlyOwner {
        require(msg.sender == owner);// 누구든 contract의 거래를 발생시키면 msg.sender 이게 owner이어야 한다 require은 항상 true가 되어야 한다
        _;
    }
    modifier onlyAfter {
        require(block.timestamp >= timeToDeposit); //require(now >= timeToDeposit); 현재시간 >= timeToDeposit = now + 10 seconds
        _;
    }
    modifier minBalance {
        require(address(this).balance>101 wei);
        _;
    }
}

Writing BankV3.sol


In [8]:
! solc-windows.exe --optimize --combined-json abi,bin BankV3.sol > BankV3.json

In [9]:
%%writefile BankV3DeployFromFile.js

var Web3 = require('web3');
var _abiBinJson = require('./BankV3.json');      // abi, 바이트 코드가 저장된 파일 읽기

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // 컨트랙 명 ['BankV3.sol:BankV3']
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      // 오류방지 SyntaxError: Unexpected token o in JSON at position 1
_bin=_abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 259210}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing BankV3DeployFromFile.js


In [10]:
!node BankV3DeployFromFile.js

- contract name:  [ 'BankV3.sol:BankV3' ]
Deploying the contract from 0xD45b6FBaB6415602651BF61720D166E543a8d414
hash: 0xe35a8bda4b2eba8b7629718ccc7a44fdcfbf8e585b1807e322cb7cb8aec2948b
---> The contract deployed to: 0x536c9769eD577F0d4eC115ec2c2Fc9f111a07dCE


(node:6272) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)


In [16]:
%%writefile BankV3Use.js

var Web3=require('web3');
var _abiBinJson = require('./BankV3.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/BankV3.sol:BankV3']
//console.log("- contract name: ", contractName); //or console.log(contractName[0]);
_abiArray=_abiBinJson.contracts[contractName].abi; //use just as read from file
//_abiArray=JSON.parse(JSON.stringify(_abi));
console.log("- ABI: " + _abiArray);
var bank = new web3.eth.Contract(_abiArray,"0x0042048e0e97BA996CA18fC7d027379ed786Af7a");
//var filter = bank.events.PrintLog(function (error, result) {  // 이벤트 알림은 하지 않는다.
//  if (!error)
//    console.log(result);
//});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    bank.methods.getBalance().call().then(console.log);
    await bank.methods.deposit(111).send({from: accounts[0], value:111});
    bank.methods.getBalance().call().then(console.log);
    await bank.methods.withdrawAll().send({from: accounts[0]});
    bank.methods.getBalance().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));    
}
doIt()

Writing BankV3Use.js
